In [250]:
import numpy as np
import math
import pandas as pd
import random
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [274]:
def split_frame(train):
    y = train['class']
    del train['class']
    return train.to_numpy(), y.replace(0,-1).to_numpy()

In [206]:
def linear(x, w, b):
    return np.sign(np.dot(w.T, x.T) + b ).astype(int)

In [ ]:
def gaussian_kernel(a, b):
    return 0

In [46]:
def custom_kernel(a,b):
    c = 0.5
    return ((np.transpose(a) * b + c) * (np.transpose(a) * b + c))

In [184]:
def test_k(a,b):
    #return np.inner(a,b).sum()
    return np.inner(a,b)

In [193]:
def linear_kernel(a,b):
    return np.dot(a,b.T)

In [67]:
def get_j(train_length, i):
    remaining = [j for j in range(0,train_length)]
    remaining.remove(i)
    return random.choice(remaining)

In [200]:
def calculate_error(a, x, y, x_set, b, kernel):
    f_xk = (a * y) * kernel(x, x_set) +  b
    return f_xk - y    

In [208]:
def calculate_w(a, x, y):
    return np.dot(x.T, np.multiply(a,y))

In [207]:
def new_calculate_error(w, xk, yk, b):
    return linear(xk, w, b) - yk     

In [69]:
def calculate_constraints(C, ai, aj, yi, yj):
    L = max(0, (ai-aj)) if (yi != yj) else max(0, (ai+aj-C))
    H = min(C, (C+aj-ai)) if (yi != yj) else min(C, ai+aj)
    return L, H

In [13]:
def calculate_eta(xi, xj ,kernel):
    eta1 = 2 * kernel(xi,xj)
    eta2 = kernel(xi,xi)
    eta3 = kernel(xj,xj)
    return eta1 - eta2 - eta3

In [174]:
def calculate_aj(aj, yj, error_i, error_j, eta, H, L):
    new_aj = aj - ((yj * (error_i - error_j)) / eta)
    if new_aj > H:
        return H
    elif new_aj < L:
        return L
    else:
        return new_aj

In [131]:
def calculate_ai(ai, yi, yj, a_j_old, aj):
    return ai + (yi*yj)*(a_j_old - aj)

In [17]:
def calculate_b1(b, error_i, yi, ai, a_i_old, xi, yj, aj, a_j_old, xj, kernel):
    return b - error_i - yi*(ai - a_i_old)*kernel(xi,xi) - yj*(aj - a_j_old)*kernel(xi,xj)

In [139]:
def calculate_b2(b, error_j, yi, ai, a_i_old, xi, xj, yj, aj, a_j_old, kernel):
    return b - error_j - yi*(ai - a_i_old)*kernel(xi,xj) - yj*(aj - a_j_old)*kernel(xj,xj)

In [277]:
def simplified_SMO(C, tolerance, max_iterations, train, kernel):
    x, y = split_frame(train)
    print(x, y)
    train_length = len(x)
    j_tol = 0.000005
    #initialize Lagrange multiplies
    a = np.zeros(train_length)
    b = 0
    number_of_iterations = 0
    while number_of_iterations < max_iterations:
        a_changes = 0
        #iterate over length of training set
        for i in range(0, train_length-1):
            #calculate the error with the current example
            j = get_j(train_length, i)
            xi = x[i,:]
            xj = x[j,:]
            w = calculate_w(a, x, y)
            #error_i = calculate_error(a[i], xi, y[i], x, b, kernel)
            error_i = new_calculate_error(w, xi, y[i], b)
            if (((y[i] * error_i) < (tolerance * -1)) and (a[i] < C)) or (((y[i] * error_i) > tolerance) and (a[i] > 0)):               
                a_j_old = a[j]
                a_i_old = a[i]
                L, H = calculate_constraints(C, a[i], a[j], y[i], y[j])
                if L==H:
                    continue                    
                eta = calculate_eta(xi,xj, kernel)
                if eta >= 0:
                    continue
                #error_j = calculate_error(a[j], xj, y[j], x, b, kernel)
                error_j = new_calculate_error(w, xj, y[j], b)
                a[j] = calculate_aj(a_j_old, y[j], error_i, error_j, eta, H, L)
                #if j change is miniscule do not need to update i
                if abs(a[j] - a_j_old) < j_tol:
                    continue
                a[i] = calculate_ai(a_i_old, y[i], y[j], a_j_old, a[j])
                if (a[i] > 0) and (a[i] < C):
                    b = calculate_b1(b, error_i, y[i], a[i], a_i_old, xi, y[j], a[j], a_j_old, xj, kernel)
                elif (a[j] > 0) and (a[j] < C):
                    b = calculate_b2(b, error_j, y[i], a[i], a_i_old, xi, xj, y[j], a[j], a_j_old, kernel)
                else:
                    b1 = calculate_b1(b, error_i, y[i], a[i], a_i_old, xi, y[j], a[j], a_j_old, xj, kernel)
                    b2 = calculate_b2(b, error_j, y[i], a[i], a_i_old, xi, xj, y[j], a[j], a_j_old, kernel)
                    b = (b1 + b2) / 2
                a_changes+=1
        #if convergence (i.e. no changes to multipliers throughout entire iteration)
        print(a_changes, number_of_iterations)
        if a_changes == 0:
            number_of_iterations+=1
        else:
            number_of_iterations=0
        idx = np.where(a > 0)[0]
        s_vec = x[idx,:]
        w = calculate_w(a,x,y)
    return a, b, s_vec,w

In [252]:
#parameters
C = 1
tolerance = 0.01
max_iterations = 10

In [278]:
blobby = pd.read_csv("blobs.csv")
del blobby["Unnamed: 0"]
train, test = train_test_split(blobby, test_size=0.3, random_state=42)
a, b, s_vec, w = simplified_SMO(C, tolerance, max_iterations, train, test_k)

[[1.99727937 2.90336564]
 [4.91706468 2.79411534]
 [4.30210045 3.6922861 ]
 ...
 [4.8222607  2.84290839]
 [4.71838251 1.97747426]
 [5.94227952 3.00314526]] [-1  1  1 ...  1  1  1]
5 0
3 0
0 0
0 1
0 2
0 3
0 4
0 5
0 6
0 7
0 8
0 9


In [279]:
b

-31.5014297216499

In [280]:
s_vec

array([[4.30210045, 3.6922861 ],
       [2.86115075, 2.79726483],
       [2.38153718, 1.74098285],
       [4.31219775, 2.86726862],
       [3.55113704, 2.47902328],
       [2.08122428, 2.4965426 ],
       [4.71325644, 3.07203531],
       [3.81095129, 2.11069187],
       [2.37230423, 1.8424646 ],
       [2.02484468, 1.74424523],
       [3.91229725, 1.95404975],
       [4.0990101 , 3.36172888],
       [5.6335694 , 2.60197824],
       [3.66669067, 2.18992106],
       [3.69848882, 2.37732636],
       [2.85263477, 1.32703912]])

In [281]:
w

array([5.94232956, 4.27994074])

In [249]:
def predict(w,x_set,b):
    predictions = [0 for i in range(0,len(x_set))]
    for i in range(0, len(x_set)):
        predictions[i] = linear(x_set[i], w, b)
    return predictions

In [ ]:
#plotting time
plt.scatter(blobs_set["0"], blobs_set["1"])
xx = np.linspace(0,7)
yy = (-w[0] / w[1]) * xx - v / w[1]